# Connections using ConceptNet

The goal is to pick a term for a category, then to pick a set of terms that all share some (perhaps the same) relation with the former.

In [77]:
import requests
import time
from random import choice, shuffle

In [175]:
RELS = (
    'PartOf',
    # 'IsA',
    # 'DerivedFrom',
    # 'CapableOf',
    'HasA',
    'Causes',
    # 'AtLocation',
    'HasProperty',
    # 'Synonym',
)

In [174]:
WORDS = [
    'time',
'year',
'people',
'way',
'day',
'man',
'thing',
'woman',
'life',
'child',
'world',
'school',
'state',
'family',
'student',
'group',
'country',
'problem',
'hand',
'part',
'place',
'case',
'week',
'company',
'system',
'program',
'question',
'work',
'government',
'number',
'night',
'point',
'home',
'water',
'room',
'mother',
'area',
'money',
'story',
'fact',
'month',
'lot',
'right',
'study',
'book',
'eye',
'job',
'word',
'business',
'issue',
'side',
'kind',
'head',
'house',
'service',
'friend',
'father',
'power',
'hour',
'game',
'line',
'end',
'member',
'law',
'car',
'city',
'community',
'name',
'president',
'team',
'minute',
'idea',
'kid',
'body',
'information',
'back',
'parent',
'face',
'others',
'level',
'office',
'door',
'health',
'person',
'art',
'war',
'history',
'party',
'result',
'change',
'morning',
'reason',
'research',
'girl',
'guy',
'moment',
'air',
'teacher',
'force',
'education',
]

In [135]:
def conceptnet(term: str, lang:str='en', all_results=True, verbose=False) -> dict:
    if verbose:
        print(f'Querying term "{term}"')
        start = time.time()
    res = requests.get(f'http://api.conceptnet.io/c/{lang}/{term}')
    if verbose:
        elapsed = time.time() - start
        print(f'Response received after {elapsed}s.')
    return res.json()

def conceptnet_query(start=None, end=None, rel=None, lang='en', verbose=False):
    base_request = 'http://api.conceptnet.io/query?'
    if start:
        base_request += f'start=/c/{lang}/{start}&'
    if end:
        base_request += f'end=/c/{lang}/{end}&'
    if rel:
        base_request += f'rel=/r/{rel}&'
    
    if verbose:
        print(f'Querying {base_request}')
        start = time.time()
    res = requests.get(base_request)
    if verbose:
        elapsed = time.time() - start
        print(f'Response received after {elapsed}s.')
    return res.json()

In [162]:
def remove_word(word, terms):
    return [term for term in terms if word not in term][:4]
    
def random_category(word, rels=RELS, verbose=False):
    if len(rels) == 0:
        raise RuntimeError('No valid relations for' + word  + '!')
    rel = choice(list(rels))
    anchor = choice(('start', 'end'))

    if anchor == start:
        res = conceptnet_query(start=word, rel=rel, verbose=verbose)
        other_anchor = 'end'
    else:
        res = conceptnet_query(end=word, rel=rel, verbose=verbose)
        other_anchor = 'start'
    
    terms = [edge[other_anchor]['label'] for edge in res['edges'] if edge[other_anchor]['language'] == 'en']
    terms = remove_word(word, terms)
    if len(terms) < 4:
        return random_category(word, rels=set(rels) - set([rel]), verbose=verbose)
    else:
        shuffle(terms)
        return word, anchor, rel, terms

In [169]:
def connections(wordlist:list=WORDS, rels=RELS, k=4, verbose=False):
    wl = wordlist.copy()
    shuffle(wl)
    cats = []
    for i in range(k):
        word = wl.pop()
        success = None
        while not success:
            try:
                success, anchor, rel, terms = random_category(word, rels, verbose=verbose)
            except RuntimeError:
                word = wl.pop()
        cats.append((word, anchor, rel, remove_word(word, terms)))
    return cats     

In [164]:
def print_category(word, anchor, rel, terms):
    if anchor == 'start':
        print(f'{word} {rel} {terms}')
    else:
        print(f'{terms} {rel} {word}')

In [172]:
game = connections(verbose=False)
for w,a,r,t in game:
    print_category(w,a,r,t) 

change CapableOf ['Weather patterns', 'humans', 'A diagnosis', 'A name']
['A bumper', 'wheels', 'A tire', 'An engine '] PartOf car
government PartOf ['the IRS', 'administration', 'judiciary', 'police']
book PartOf ['signature', 'pages', 'A chapter', 'page']
